# Parkinson's Diagnose

In [3]:
import os
os.chdir('/Users/hanliyang/Documents/GitHub/Parkinson-Prediagnosis')

## Audio Conversion
* Using assembly AI

In [11]:
import assemblyai as aai

aai.settings.api_key = "d9ccd193e5e54a4ca8bdddaf3769bc59"

config = aai.TranscriptionConfig(speaker_labels=True)

transcriber = aai.Transcriber()

audioPath = ""

def audioTextConversion(audioPath):
    dialogue = {
        "Speaker A" : [],
        "Speaker B" : []
    }
    transcript = transcriber.transcribe(
    audioPath,
    config=config
    )

    for utterance in transcript.utterances:
        print(f"Speaker {utterance.speaker}: {utterance.text}")
        speaker = f"Speaker {utterance.speaker}"
        dialogue[speaker].append(utterance.text)
    return speaker_differentiation(dialogue)
    
def speaker_differentiation(dialogue):
    totalA = 0
    totalB = 0
    for text in dialogue['Speaker A']:
        totalA += len(text)
    
    for text in dialogue['Speaker B']:
        totalB += len(text)

    if totalA > totalB:
        return {'participant':str(dialogue["Speaker A"]), 
                "interviewer": str(dialogue["Speaker B"])} 
    else:
        return {'participant':str(dialogue["Speaker B"]), 
                "interviewer": str(dialogue["Speaker A"])} 
 

In [12]:
import pandas as pd
dialogue = audioTextConversion('NLP/Data/SpontaneousDialogue/HC/ID00_hc_0_0_0.wav')

df = pd.DataFrame(data = dialogue)


KeyError: 'Spaeaker A'

## Counting Pos

In [10]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize

tokens = "Computer science is the study of computation, automation, and information."\
"Computer science spans theoretical disciplines (such as algorithms, theory of computation, information theory, and automation) to practical disciplines (including the design and implementation of hardware and software)."\
"Computer science is generally considered an academic discipline and distinct from computer programming."

[nltk_data] Downloading package punkt to /Users/hanliyang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/hanliyang/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


* verbs - VB	verb (ask)
VBG	verb gerund (judging)
VBD	verb past tense (pleaded)
VBN	verb past participle (reunified)
VBP	verb, present tense not 3rd person singular(wrap)
VBZ	verb, present tense with 3rd person singular (bases)
* common nouns - NN, NNS
* proper nouns - NNP, NNPS


In [11]:
filler_words = [
    "um", "uh", "er", "like", "you know", "i mean", "well", "so", "okay", "right",
    "actually", "basically", "literally", "kind of", "sort of", "anyway", "or something",
    "whatever", "i guess", "you see", "got it", "i suppose", "just", "yeah", "yep",
    "seriously", "alright", "hmm", "look", "totally", "alright then", "know what i mean?",
    "at the end of the day", "to be honest", "if you know what i mean"
]

def pdFeaturesCount(tokens):
  wordList = pos_tag(word_tokenize(tokens))
  totalWords, verbsRate, verbs, commonNouns, properNouns, fillers = 0, 0, 0, 0, 0, 0
  for word, pos in wordList:
    #print(word, pos)
    totalWords += 1
    if pos in ['VB', 'VBG', 'VBD', 'VBN', 'VBP', 'VBZ']:
      verbs += 1
    if pos in ['NN', 'NNS']:
      commonNouns += 1
    if pos in ['NNP', 'NNPS']:
      properNouns += 1
    if word in filler_words:
      fillers += 1
  verbsRate = verbs / totalWords
  return totalWords, verbsRate, verbs, commonNouns, properNouns, fillers

In [21]:
pdFeaturesCount(tokens)

(59, 0.06779661016949153, 4, 25, 1, 0)